In [1]:
import os, sys
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
matplotlib.rc('font', family='NanumBarunGothic') # Linux
plt.rcParams['axes.unicode_minus'] = False

def plot_image(image, shape=[28, 28]):
    plt.imshow(image.reshape(shape), cmap="Greys", interpolation="nearest")
    plt.axis("off")
    
def plot_multiple_images(images, n_rows, n_cols, pad=2):
    images = images - images.min()  # 최소값을 0으로 만들어 패딩이 하얗게 보이도록 합니다.
    w,h = images.shape[1:]
    image = np.zeros(((w+pad)*n_rows+pad, (h+pad)*n_cols+pad))
    for y in range(n_rows):
        for x in range(n_cols):
            image[(y*(h+pad)+pad):(y*(h+pad)+pad+h),(x*(w+pad)+pad):(x*(w+pad)+pad+w)] = images[y*n_cols+x]
    plt.imshow(image, cmap="Greys", interpolation="nearest")
    plt.axis("off")

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
    
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 1000  # 희소 코딩 유닛
n_outputs = n_inputs

In [7]:
def kl_divergence(p, q):
    # 쿨백 라이블러 발산
    return p * tf.log(p / q) + (1 - p) * tf.log((1 - p) / (1 - q))

learning_rate = 0.01
sparsity_target = 0.1
sparsity_weight = 0.2

X = tf.placeholder(tf.float32, shape=[None, n_inputs])            # 책에는 없음

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.sigmoid) # 책에는 없음
outputs = tf.layers.dense(hidden1, n_outputs)                     # 책에는 없음

hidden1_mean = tf.reduce_mean(hidden1, axis=0) # 배치 평균
sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, hidden1_mean))
reconstruction_loss = tf.reduce_mean(tf.square(outputs - X)) # MSE
loss = reconstruction_loss + sparsity_weight * sparsity_loss

optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

In [8]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [9]:
n_epochs = 100
batch_size = 1000

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = len(X_train) // batch_size
        for iteration in range(n_batches):
            print("\r{}%".format(100 * iteration // n_batches), end="")
            sys.stdout.flush()
            X_batch, y_batch = next(shuffle_batch(X_train, y_train, batch_size))
            sess.run(training_op, feed_dict={X: X_batch})
        reconstruction_loss_val, sparsity_loss_val, loss_val = sess.run([reconstruction_loss, sparsity_loss, loss], feed_dict={X: X_batch})
        print("\r{}".format(epoch), "훈련 MSE:", reconstruction_loss_val, "\t희소 손실:", sparsity_loss_val, "\t전체 손실:", loss_val)
        saver.save(sess, "./my_model_sparse.ckpt")

0 훈련 MSE: 0.06617325 	희소 손실: 0.003382287 	전체 손실: 0.06684971
1 훈련 MSE: 0.063242674 	희소 손실: 5.8615464e-05 	전체 손실: 0.06325439
2 훈련 MSE: 0.061897162 	희소 손실: 0.00038104644 	전체 손실: 0.06197337
3 훈련 MSE: 0.06215215 	희소 손실: 3.0237716e-05 	전체 손실: 0.062158197
4 훈련 MSE: 0.059958447 	희소 손실: 0.0012078471 	전체 손실: 0.060200017
5 훈련 MSE: 0.05795762 	희소 손실: 0.0014181416 	전체 손실: 0.058241248
6 훈련 MSE: 0.058964625 	희소 손실: 0.00030663516 	전체 손실: 0.05902595
7 훈련 MSE: 0.05805256 	희소 손실: 0.0007861671 	전체 손실: 0.05820979
8 훈련 MSE: 0.05846902 	희소 손실: 6.5569766e-05 	전체 손실: 0.058482133
9 훈련 MSE: 0.05857562 	희소 손실: 0.00033814833 	전체 손실: 0.058643248
10 훈련 MSE: 0.05838547 	희소 손실: 6.2436855e-05 	전체 손실: 0.058397956
11 훈련 MSE: 0.058195334 	희소 손실: 0.000335366 	전체 손실: 0.058262408
12 훈련 MSE: 0.05785378 	희소 손실: 0.00026819902 	전체 손실: 0.05790742
13 훈련 MSE: 0.05748144 	희소 손실: 0.00047850236 	전체 손실: 0.05757714
14 훈련 MSE: 0.057812005 	희소 손실: 0.0006149863 	전체 손실: 0.057935003
15 훈련 MSE: 0.05774809 	희소 손실: 2.44088e-05 	전체 손실: 0.0577529